In [1]:
import sys
sys.path.append('/scratch/ne2213/projects/tmp_packages')
sys.path.append('/scratch/ne2213/projects/tmp_packages/')

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient
import h5py
import tempfile
import requests
import warnings
warnings.filterwarnings("ignore", message="Ignoring cached namespace")

In [3]:
# Define DANDI IDs and output directory
dandi_ids = {
    "EY": "000541",
    "SK1": "000565",
    "NP": "000715",
    "SK2": "000472",
    "HL": "000714",
    "KK": "000692",
    "SF": "000776"
}

# Define the base output directory
base_output_dir = "/scratch/ne2213/projects/CV-Project/NWBelegans/extracted_data_all/"

# Ensure the directory exists
os.makedirs(base_output_dir, exist_ok=True)

In [ ]:
# Process each dataset
for label, dandi_id in dandi_ids.items():
    print(f"Processing dataset {label} with DANDI ID {dandi_id}...")

    # Initialize lists to store NeuroPAL data
    neuron_voxel_masks = []
    neuron_labels = []
    grid_spacing = []
    rgb_values = []  # Store RGB values for each voxel
    structured_data = []  # Store full structured data in a DataFrame

    with DandiAPIClient() as client:
        print(f"Connecting to DANDI API for {label}...")
        
        # Get the dandiset
        dandiset = client.get_dandiset(dandi_id, 'draft')
        print(f"Retrieved dandiset {dandi_id} for {label}.")

        # Iterate through the assets in the dandiset
        for asset in dandiset.get_assets():
            print(f"Processing asset: {asset.identifier}...")

            # Get the asset URL for downloading
            s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

            # Create a temporary file to store the downloaded asset
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file_path = temp_file.name
                print(f"Downloading asset {asset.identifier} to temporary file...")
                
                # Download the asset using requests
                response = requests.get(s3_url, stream=True)
                if response.status_code == 200:
                    for chunk in response.iter_content(chunk_size=8192):
                        temp_file.write(chunk)
                    print(f"Download complete for {asset.identifier}.")
                else:
                    print(f"Failed to download {s3_url}. Status code: {response.status_code}")
                    continue

            # Open the NWB file and extract data
            try:
                print(f"Opening NWB file: {temp_file_path}...")
                with h5py.File(temp_file_path, 'r') as f:
                    with NWBHDF5IO(file=f, mode='r', load_namespaces=True) as io:
                        read_nwb = io.read()
                        print(f"Successfully read NWB file for {asset.identifier}.")

                        # Extract NeuroPAL data
                        try:
                            seg = read_nwb.processing['NeuroPAL']['NeuroPALSegmentation']['NeuroPALNeurons'].voxel_mask[:]
                            labels = read_nwb.processing['NeuroPAL']['NeuroPALSegmentation']['NeuroPALNeurons']['ID_labels'][:]
                            channels = read_nwb.acquisition['NeuroPALImageRaw'].RGBW_channels[:]
                            image = read_nwb.acquisition['NeuroPALImageRaw'].data[:]
                            scale = read_nwb.imaging_planes['NeuroPALImVol'].grid_spacing[:]

                            # Process labels into strings
                            labels = ["".join(label) for label in labels]
                            print(f"Extracted NeuroPAL data for {asset.identifier}.")

                            # Create a structured DataFrame for the voxel data
                            blobs = pd.DataFrame.from_records(seg, columns=['x', 'y', 'z', 'weight'])
                            blobs = blobs[
                                (blobs['x'] < image.shape[0]) &
                                (blobs['y'] < image.shape[1]) &
                                (blobs['z'] < image.shape[2])
                            ]

                            # Add RGB values
                            blobs[['R', 'G', 'B']] = [
                                image[int(row['x']), int(row['y']), int(row['z']), channels[:3]]
                                for _, row in blobs.iterrows()
                            ]

                            # Add scaled spatial positions
                            blobs[['xr', 'yr', 'zr']] = [
                                [row['x'] * scale[0], row['y'] * scale[1], row['z'] * scale[2]]
                                for _, row in blobs.iterrows()
                            ]

                            # Add labels to the blobs
                            blobs['ID'] = labels[:len(blobs)]  # Align the labels with blobs

                            # Append to lists for saving
                            neuron_voxel_masks.append(seg)
                            neuron_labels.append(labels)
                            grid_spacing.append(scale)
                            rgb_values.append(blobs[['R', 'G', 'B']].values.tolist())
                            structured_data.append(blobs)

                            print(f"Processed structured data for {asset.identifier}.")
                        except KeyError as e:
                            print(f"KeyError while processing {asset.identifier}: {e}. Skipping this asset.")

            except Exception as e:
                print(f"Error processing file {temp_file_path}: {e}")

            # Remove the temporary file
            os.remove(temp_file_path)
            print(f"Temporary file {temp_file_path} removed.")

    # Combine all data into a dictionary
    print(f"Combining all data for {label}...")
    data = {
        "neuron_voxel_masks": neuron_voxel_masks,
        "neuron_labels": neuron_labels,
        "grid_spacing": grid_spacing,
        "rgb_values": rgb_values,
        "structured_data": pd.concat(structured_data, ignore_index=True) if structured_data else None
    }

    # Save all data into a single .pkl file
    output_file = os.path.join(base_output_dir, f"{label}_data.pkl")
    with open(output_file, "wb") as f:
        pickle.dump(data, f)
    print(f"Finished saving all data for {label} in {output_file}.\n")

print("All datasets processed.")